# Sberbank version of AutoML (AI Lab production) 

Данное решение представляет собой Python библиотеку, позволяющую автоматически обучать модель и проверять их качество на тестовых датасетах.

**!!!! Данная версия не имеет обратной совместимости с версиями ранее 1.3.0, комментарии см. ниже**

**V1.3.0. Обновления по сравнению с предыдущей версией:**

1. Отсутсвует аргумент model_dir при создании AutoML. При тренировке и инференсе на диск ничего не сохраняется. Исключение - если указана cahce_dir
2. Появилась возможность указаня cache_dir для train/predict. Аргумент не обязателен, но повышает эффективность обработки текстов. Эту директорию можно удалить по завершении работы, а можно оставить для ускорения повторных экспериментов на тех же данных. 
3. Появилась возможность обучать модели из памяти (передавать DataFrame) без сохранения его в csv. Интерфейс точно такой же, только вместо пути до файла можно передать другой объект
4. Появилась возможность ускорять инференс за счет параллельного выполнения. Аргумент - n_jobs
5. Появился sklearn подобный интерфейс (см. Package_usage_sklearn)
6. Для сохранения и загрузки моделей методам .save и .load теперь надо передавать путь до файла модели. Так же теперь возможно сохранение при помощи Pickle 

Еще раз отмечу здесь **важное замечание**, которое необходимо учитывать при работе с библиотекой: в случае, если используемый для тренировки датасет не помешается в заданную память, в логе будет выведено сообщение "We are sampling the data to fit the memory" - это означает, что датасет для непосредственной тренировки моделей был семплирован из исходного для того, чтобы уместиться в заданную память. 

При решении задачи регрессии можно полностью игнорировать данный факт (либо поставить больше памяти чтобы использовать больше данных для обучения).

При решении задачи классификации данный факт можно игнорировать в случае, если необходимо просто оценить метрику полученной модели (Gini, AUC и т.п.). Если планируется использовать полученные из модели вероятности в дальнейшем бизнес процессе, то **необходимо самостоятельно сделать для них калибровку**.

In [ ]:
import time
start_time = time.time()

# Package installation 

Библиотеку можно установить для текущего юзера следующей командой:

In [ ]:
!pip uninstall --yes sber_ailab_automl 
!python setup.py install --user

# Necessary imports 

In [ ]:
import sber_ailab_automl as saa
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score, mean_squared_error
import logging
import sys
logging.basicConfig(level=logging.INFO, stream=sys.stdout,
                        format='[%(asctime)-15s] %(levelname)-8s: %(message)s')
logger = logging.getLogger(__name__)

In [ ]:
saa.__version__

# Set parameters 

Опишем те параметры, которые можно передать решению для работы:

- `params['vCPULimit']` - ограничение по количеству используемых виртуальных ядер процессора
- `params['memoryLimit']` - ограничение на используемую память
- `params['timeLimit']` - ограничение на время работы
- `params['type']` - тип задачи ('classification' или 'regression')
- `params['trainDatasetFName']` - путь до файла с тренировочной выборкой 
- `params['testDatasetFName']` - путь до файла с тестовой выборкой
- `params['predictionsFName']` - путь до файла с итоговыми предсказаниями
- `params['encoding']` - кодировка, в которой сохранены тренировочный и тестовый файлы (для Unix систем это обычно `'utf-8'`, а для Windows - `'cp1251'`)
- `params['separator']` - разделитель полей в CSV файле (обычно `r','`, а для файла, экспортированного из Teradata - `r'\t'`). Букву r перед параметром необходимо сохранить для правильного чтения параметра (в сам параметр он не входит)
- `params['decimal']` - разделитель целой и дробной части для вещественных чисел (обычно `'.'`, а для файла, экспортированного из Teradata - `','`)
- `params['debugFlag']` - флаг debug режима (если True, то печатается лог тренировки; если False - то нет)
- `params['featureRoles']` - параметр, в котором указываются роли и типы переменных в виде словаря:
    * `target` - строка с названием колонки целевой переменной (**обязательный параметр**)
    * `line_id` - строка с названием колонки id строки (необязательный параметр)
    * `group` - строка с названием колонки-группы для объекта, используется для GroupKFold валидации (необязательный параметр). **Важное замечание**: пока что поддерживается только одна колонка в качестве группы, а также данная колонка НЕ должна содержать пропущенных значений
    * `drop` - массив названий колонок, которые необходимо удалить из датасета (необязательный параметр)
    * `numeric` - массив названий колонок, которые являются вещественными признаками (необязательный параметр)
    * `datetime` - массив названий колонок, которые являются датами (в указанном в параметре `params['datetimeFormat']` формате) (необязательный параметр)
    * `string` - массив названий колонок, которые являются строками (категориальными признаками) (необязательный параметр)
    * `id` - массив названий колонок, которые являются id-шниками (необязательный параметр)
    * `text` - массив названий колонок, которые являются текстами (необязательный параметр)
- `params['naValues']` - символ пропущенного значения (`None` если пропуски в CSV файле являются пустыми)
- `params['rowsToAnalyze']` - количество строчек, по которым автоматически определяются типы данных
- `params['datetimeFormat']` - формат, в котором в файле записаны даты (более подробно о форматах можно посмотреть [здесь](https://docs.python.org/3/library/datetime.html#strftime-strptime-behavior))
- `params['useGPU']` - флаг использования GPU в задаче с текстовыми данными (будет работать только при подходящей под установленную CUDA версии torch)
- `params['testBatchSize']` - размер батча, используемый для предсказания тестовой выборки
- `params['LuckySeed']` - фиксируемый seed для построения воспроизводимой кросс-валидации (также может быть использован при построении ансамбля AutoML моделей, построенных на разных CV разбиениях)
- `params['KFolds']` - количество частей, на которые разбивается выборка в процессе решения задачи
- `params['metric']` - метрика, которую необходимо оптимизировать:
    * `None` - используются дефолтные метрики AutoML (AUC для классификации и RMSE для регрессии)
    * `Func` - для кастомной метрики, которая определена функцией `Func(y_true, y_pred)`
- `params['featureSelParams']` - параметры для использования отбора признаков внутри AutoML, используется нотация tuple:
    * Первый элемент tuple отвечает за включение/отключение отбора признаков, а также за максимальное количество признаков, которые нужно использовать в модели (примеры значений: `None` - отбор признаков выключен; `-1` - набор признаков включен, количество выбираемых признаков не ограничено;  `N > 0` - отбор включен, при отборе оставить не более N признаков)
    * Второй элемент tuple отвечает за размер группы признаков, одновременно проверяемых в процессе отбора (может быть от 1 до +inf). Чем меньше этот параметр, тем более тщательно будут отбираться признаки, но тем дольше это будет происходить, поэтому необходимо выбирать trade-off скорости/результата для конкретного датасета.
    * Третий элемент tuple отвечает за отбрасывание признаков по Permutation Importance, которые ухудшают качество модели (отрицательный Permutation Importance при использовании метрики, которую нужно максимизировать, и положительный в противном случае), перед отбором признаков.

In [ ]:
params = {}
params['vCPULimit'] = 20
params['memoryLimit'] = 20 * 1024
params['timeLimit'] = 3000 * 60
params['type'] = 'regression'
params['trainDatasetFName'] = 'example_data/check_1_r/train.csv'
params['testDatasetFName'] = 'example_data/check_1_r/test.csv'
params['predictionsFName'] = 'preds.csv'
params['encoding'] = 'utf-8'
params['separator'] = r","
params['decimal'] = '.'
params['debugFlag'] = True
params['featureRoles'] = {
    'target': 'target',
    'line_id': 'line_id',
    #'group': 'col_name',
    'drop': [],
    'numeric': [],
    'datetime': [],
    'string': [],
    'id': [],
    'text': []
}
params['naValues'] = None
params['rowsToAnalyze'] = 10000
params['datetimeFormat'] = '%Y-%m-%d'
params['testBatchSize'] = 10000
params['LuckySeed'] = 42
params['KFolds'] = 5
params['metric'] = None 
params['featureSelParams'] = (-1, 5, True)

# Train and save model 

В ячейке ниже производится объявление модели AutoML, ее обучение и сохранение. Все необходимые параметры задаются в ячейке с параметрами.

In [ ]:
%%time
automl = saa.AutoML(vCPULimit = params['vCPULimit'], 
                    memoryLimit = params['memoryLimit'], 
                    timeLimit = params['timeLimit'], 
                    encoding = params['encoding'], 
                    separator = params['separator'], 
                    decimal = params['decimal'], 
                    datetime_format = params['datetimeFormat'], 
                    na_values = params['naValues'], 
                    analyze_rows = params['rowsToAnalyze'],
                    test_batch_size = params['testBatchSize'],
                    cv_random_state = params['LuckySeed'],
                    KFolds = params['KFolds'],
                    metric = params['metric'],
                    featureSelParams = params['featureSelParams'])

res_df, feat_imp = automl.train(params['trainDatasetFName'], params['featureRoles'], params['type'], )
automl.save('new_pickle_way.pkl')

# Load trained model and predict

В ячейке ниже производится объявление модели AutoML, ее загрузка и предсказание целевой переменной для тестового файла. Все необходимые параметры также задаются в ячейке с параметрами.

In [ ]:
%%time
automl = saa.AutoML()

automl.load('new_pickle_way.pkl')
preds_df, score, res_df_test = automl.predict(params['testDatasetFName'], params['predictionsFName'],   n_jobs = 4
                                             )

# Show the model score on test dataset 

Если колонка с целевой переменной присутствует в тестовом датасете, то скор модели будет автоматически записан в переменную `score`. Поскольку в использованном тестовом датасете отсутствует колонка с целевой переменной, то необходимо вычислить скор самостоятельно.

In [ ]:
logging.info('Score is {:.3f}'.format(score))

In [ ]:
logging.info('This notebook took {:.3f} hours'.format((time.time() - start_time) / 3600))

# Example of averaging several AutoML models 

Ниже приведен пример построения усредненной композиции 5 моделей AutoML, обученных по различным кросс-валидациям:

In [ ]:
def update_probs(res, cols, N_Starts, new_res = None):
    if new_res is None:
        for col in cols:
            res[col] /= N_Starts
    else:
        for col in cols:
            res[col] += new_res[col].values / N_Starts
    return res

N_Starts = 10
            
for it, seed in enumerate(range(N_Starts)):
    logging.info('START {} ITERATION WITH SEED = {} '.format(it, seed) + '*' * 30)
    params['LuckySeed'] = seed
    automl = saa.AutoML(vCPULimit = params['vCPULimit'], 
                        memoryLimit = params['memoryLimit'], 
                        timeLimit = params['timeLimit'], 
                        encoding = params['encoding'], 
                        separator = params['separator'], 
                        decimal = params['decimal'], 
                        datetime_format = params['datetimeFormat'], 
                        na_values = params['naValues'], 
                        analyze_rows = params['rowsToAnalyze'],
                        test_batch_size = params['testBatchSize'],
                        cv_random_state = params['LuckySeed'],
                        KFolds = params['KFolds'],
                        metric = params['metric'],
                       featureSelParams = params['featureSelParams'])

    res_df, feat_imp = automl.train(params['trainDatasetFName'], params['featureRoles'], params['type'], use_ids = False)
    preds_df, score, res_df_test = automl.predict(params['testDatasetFName'], params['predictionsFName'], n_jobs = 1)
    
    if it == 0:
        oof_predictions = update_probs(res_df.copy(), ['REG', 'GBM', 'ENS'], N_Starts)
        if score is not None:
            test_predictions = update_probs(res_df_test.copy(), ['REG', 'GBM', 'ENS'], N_Starts)
            scores = [score]
        else:
            test_predictions = update_probs(preds_df.copy(), ['prediction'], N_Starts)
    else:
        oof_predictions = update_probs(oof_predictions, ['REG', 'GBM', 'ENS'], N_Starts, res_df)
        if score is not None:
            test_predictions = update_probs(test_predictions, ['REG', 'GBM', 'ENS'], N_Starts, res_df_test)
            scores.append(score)
        else:
            test_predictions = update_probs(test_predictions, ['prediction'], N_Starts, preds_df)

In [ ]:
if params['type'] == 'regression':
    metric = lambda y_true, y_pred: mean_squared_error(y_true, y_pred) ** 0.5
else:
    metric = lambda y_true, y_pred: roc_auc_score(y_true, y_pred)

oof_score_avg = metric(oof_predictions.TRUE.values, oof_predictions.ENS.values)
logging.info('OOF score of avg_composition is {:.3f}'.format(oof_score_avg))

if score is not None:
    logging.info('Test scores on runs = {}'.format(scores))
    test_score_avg = metric(test_predictions.TRUE.values, test_predictions.ENS.values)
    logging.info('Test score of avg_composition is {:.3f}'.format(test_score_avg))

Посмотрим как выглядит распределение результатов при различных запусках:

In [ ]:
pd.Series(scores).describe()